In [4]:
import sys
sys.path.append('./')
import numpy as np
import pandas as pd
import neural_tape_controller
#Положительные числа - положительные награды.
import tasks 
import optimize
import pickle
root_dir = './'

In [5]:
nt = neural_tape_controller.nt_controller(tacts=1,input_size=36,output_size=5)
genom = nt.nn.disassemble_genom()

In [6]:
try:
    with open(root_dir+f'./genoms/best_genoms_aa_gun.pkl', 'rb') as f:
        genom = pickle.load(f)[-1]
    print('loaded successfully')
except Exception:
    pass

In [7]:
%%time
q=tasks.custom_test(genom, verbose=True, draw=False)
print('q=',q)

aa_gun_2: [0 0 0 0 0 0]  full=0.0
q= -1.0067224502563477e-13
Wall time: 2.52 s


In [8]:
print(pd.Timestamp.now())

2022-05-11 20:46:34.775854


In [9]:
%%time
n_jobs = 7
from multiprocessing import Pool
pool = Pool(processes=n_jobs)
y_lst = pool.map(tasks.custom_test, [x for x in [genom]*n_jobs])
pool.close()
pool.join()

Wall time: 8.09 s


In [10]:
print(1)
opt = optimize.optimizer(tasks.custom_test, genom_size=len(genom),parallel_cores=2)
try:
    with open(root_dir+f'./genoms/best_genoms_aa_gun.pkl', 'rb') as f:
        opt.best_genoms = pickle.load(f)
    print('loaded successfully')
except Exception:
    pass
opt.function(opt.best_genoms[-1], verbose=True)

for i in range(1000):
    print('opt#',i)
    opt.optimize()
    with open(root_dir+f'./genoms/best_genoms_aa_gun.pkl', 'wb') as f:
        pickle.dump(opt.best_genoms,f,protocol=pickle.HIGHEST_PROTOCOL)
        print('WRITTEN')

1
aa_gun_2: [0 0 0 0 0 0]  full=0.0
opt# 0
scores for optimizers augmented [1.e+10 1.e+10 1.e+10 1.e+10 1.e+10 1.e+10 1.e+10 1.e+10]
chosen evol_wide previous_result: nan per tacts: nan


C:\Users\kilorad\PYTHON\neuroevolution_toolbox_\optimize.py:36: RuntimeWarning: Mean of empty slice
  mx.append(np.nanmean(self.history_gain[opt_name])-time_penalty*np.nanmean(self.history_time[opt_name]))
C:\Users\kilorad\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


iteration 0 y= [2.00033566 1.33352802 1.0003998 ]
iteration 1 y= [2.66765043 2.42192523 2.08541786]
iteration 2 y= [10.66711925 10.66704468  2.66765043]
iteration 3 y= [10.66711925 10.66704468 10.66701134]
iteration final y= [10.66711925 10.66704468 10.66701134]
result evol_wide previous_gain: nan per tacts: nan duration 0 days 00:08:16.583584
aa_gun_2: [20.0031 10.003  10.0021 10.     20.002  10.0021]  full=10.667119248176668
WRITTEN
opt# 1
scores for optimizers augmented [8.57678359e+00 1.00000000e+10 1.00000000e+10 1.00000000e+10
 1.00000000e+10 1.00000000e+10 1.00000000e+10 1.00000000e+10]
chosen evol_mid_chaos previous_result: nan per tacts: nan
iteration 0 y= [10.66711925 10.33377743 10.33377743]


KeyboardInterrupt: 

In [ ]:
#draw
with open(root_dir+f'./genoms/best_genoms_aa_gun.pkl', 'rb') as f:
    genom = pickle.load(f)
    genom = genom[-1]
print('loaded successfully')
q=tasks.custom_test(genom, verbose=True, draw=True)
print('q=',q)

In [ ]:
#save gif
tasks.video[0].save(
    root_dir+'./out_videos/best_genoms_aa_gun.gif',
    save_all=True,
    append_images=tasks.video[1:], 
    optimize=True,
    duration=100,
    loop=0
)

In [102]:
#draw video
import cv2
writer = cv2.VideoWriter("outputvideo.mp4",cv2.VideoWriter_fourcc(*"MP4V"),30,(512,512))
for i in range(0,len(tasks.video),1):
    resize = cv2.resize(np.array(tasks.video[i]), (512, 512))
    frame = resize*0
    frame[:,:,0],frame[:,:,1],frame[:,:,2] = resize[:,:,2],resize[:,:,1], resize[:,:,0]
    writer.write(frame)
writer.release()